In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [ ]:
valid_acc1=[0.625,0.500,0.625,0.500,0.625,0.875,1.000,1.000,0.750,1.000,1.000,0.875,1.000,1.000,1.000]
valid_auc1=[0.516,0.594,0.734,0.500,0.766,0.953,1.000,1.000,0.938,1.000,1.000,0.953,1.000,1.000,1.000]
valid_loss1=[1.094, 0.710,0.632,0.894,0.515,0.351,0.317,0.262,0.254,0.165,0.288,0.273,0.078,0.194,0.140]

valid_acc2=
valid_auc2=
valid_loss2=

valid_acc3=
valid_auc3=
valid_loss3=

valid_acc4=
valid_auc4=
valid_loss4=

valid_acc5=
valid_auc5=
valid_loss5=

test_acc=[0.909]
test_auc=[0.986]